In [22]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup

In [3]:
exec_orders = pd.read_csv('exec_orders_w_text')

In [7]:
exec_orders[exec_orders.order_text.isnull()].groupby('president').count()

,Unnamed: 0,citation,document_number,end_page,html_url,pdf_url,type,subtype,publication_date,signing_date,start_page,title,disposition_notes,executive_order_number,order_text
president,,,,,,,,,,,,,,,
barack_obama,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
william_j_clinton,202,202,202,202,202,201,202,202,202,202,202,202,151,201,0


In [16]:
null_orders = exec_orders[exec_orders.order_text.isnull()]

In [9]:
clinton = '/Users/lilygrier/Documents/Grad_School/Computational Content Analysis/documents_signed_by_william_j_clinton_of_type_presidential_document_and_of_presidential_document_type_executive_order.csv'

In [11]:
clinton = pd.read_csv(clinton)

In [13]:
def gen_text(html_link):
    html = requests.get(html_link)
    soup = BeautifulSoup(html.text)
    xml_links = []
    for link in soup.body.findAll('a', href=re.compile(r'^https://www.federalregister.+.xml?')):
        #if link['href'].endswith('.xml'):
        #print(link['href'])
        xml_links.append(link['href'])
        #print(type(link['href']))
    if not xml_links:
        print('no xml links found')
        print('associated html link is: ', html_link)

    elif len(xml_links) > 1:
        print('more than one xml link')
        print('associated html link is: ', html_link)
    else:
        return get_xml_text(xml_links[0])

In [14]:
def get_xml_text(xml_link):
    txt = requests.get(xml_link)
    txt.encoding = txt.apparent_encoding
    soup = BeautifulSoup(txt.text)    # txt is simply the a string with your XML file
    return soup.get_text()

In [24]:
html_link = null_orders.loc[622].html_url

In [25]:
gen_text(html_link)

no xml links found
associated html link is:  https://www.federalregister.gov/documents/1999/11/23/99-30687/amendment-to-executive-order-12958classified-national-security-information


In [26]:
html = requests.get(html_link)
soup = BeautifulSoup(html.text)

In [33]:
soup.body.findAll('a', href=re.compile(r'^https://www.govinfo.+.htm?'))

[<a href="https://www.govinfo.gov/content/pkg/FR-1999-11-23/html/99-30687.htm" target="blank">a basic text format</a>]

In [40]:
def gen_text_no_xml(html_link):
    '''
    Retrieves text from older executive orders that don't have associated xml files.
    Returns a strong with the text of the executive order.
    Inputs:
        html_link: a string containing the html link.
    '''
    html = requests.get(html_link)
    soup = BeautifulSoup(html.text)
    htm_links = []
    for link in soup.body.findAll('a', href=re.compile(r'^https://www.govinfo.+.htm?')):
        #print(link['href'])
        htm_links.append(link['href'])
        #print(type(link['href']))
    if not htm_links:
        print('no htm links found')
        print('associated html link is: ', html_link)

    elif len(htm_links) > 1:
        print('more than one htm link')
        print('associated html link is: ', html_link)
    else:
        return get_xml_text(htm_links[0])

In [38]:
gen_text_no_xml(html_link)

"\n\nFederal Register, Volume 64 Issue 225 (Tuesday, November 23, 1999)\n\n[Federal Register Volume 64, Number 225 (Tuesday, November 23, 1999)]\n[Presidential Documents]\n[Pages 66089-66090]\nFrom the Federal Register Online via the Government Publishing Office [www.gpo.gov]\n[FR Doc No: 99-30687]\n\n\n \n \n                         Presidential Documents \n \n \n\n  Federal Register / Vol. 64, No. 225 / Tuesday, November 23, 1999 / \nPresidential Documents  \n\n[[Page 66089]]\n\n\n                Executive Order 13142 of November 19, 1999\n\n                \nAmendment To Executive Order 12958--Classified \n                National Security Information\n\n                By the authority vested in me as President by the \n                Constitution and the laws of the United States of \n                America, and in order to extend and establish specific \n                dates for the time within which all classified \n                information contained in records more than 2

In [39]:
exec_orders.loc[0].order_text

"\n\nTitle 3—\nThe President\n\nExecutive Order 13764 of January 17, 2017\nAmending the Civil Service Rules, Executive Order 13488, and Executive Order 13467 To Modernize the Executive Branch-Wide Governance Structure and Processes for Security Clearances, Suitability and Fitness for Employment, and Credentialing, and Related Matters\nBy the authority vested in me as President by the Constitution and the laws of the United States of America, and as part of continuing efforts to modernize the overarching executive branch enterprise to ensure that all persons performing work for or on behalf of the Government are and continue to be loyal to the United States, reliable, trustworthy, and of good conduct and character, and by using mutually consistent standards and procedures, it is hereby ordered as follows:\n\nSection 1. Amendments to the Civil Service Rules. (a) Civil Service Rule II is amended as follows:\n(i) The title to 5 CFR Part 2 is revised to read as follows:\n“PART 2—APPOINTMENT

In [41]:
dest_file = '/Users/lilygrier/Documents/Grad_School/Computational Content Analysis/early_clinton_orders.csv'

In [42]:
null_orders.loc[:, 'order_text'] = null_orders.html_url.apply(lambda x: gen_text_no_xml(x))
#null_orders.to_csv(dest_file)

more than one htm link
associated html link is:  https://www.federalregister.gov/documents/1998/05/19/98-13552/federalism
more than one htm link
associated html link is:  https://www.federalregister.gov/documents/1997/02/14/97-3992/advisory-committee-on-high-performance-computing-and-communications-information-technology-and-the


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [43]:
null_orders.to_csv(dest_file)

In [45]:
not_null = exec_orders[exec_orders.order_text.notnull()]

In [46]:
null_orders.president

216         barack_obama
620    william_j_clinton
621    william_j_clinton
622    william_j_clinton
623    william_j_clinton
             ...        
817    william_j_clinton
818    william_j_clinton
819    william_j_clinton
820    william_j_clinton
821    william_j_clinton
Name: president, Length: 203, dtype: object

In [49]:
# now add Biden orders (as of March 7, 2021)
filename = '/Users/lilygrier/Documents/Grad_School/Computational Content Analysis/documents_signed_by_joe_biden_of_type_presidential_document_and_of_presidential_document_type_executive_order.csv'

In [50]:
biden_df = pd.read_csv(filename, index_col=None, header=0)
biden_df['president'] = re.search(r'_by_(\w+)_of_type', filename)[1] # put president name in df
#big_df = pd.concat(df_list, axis=0, ignore_index=True)


In [51]:
biden_df

,citation,document_number,end_page,html_url,pdf_url,type,subtype,publication_date,signing_date,start_page,title,disposition_notes,executive_order_number,president
0,86 FR 11849,2021-04280,11854,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-03...,Presidential Document,Executive Order,03/01/2021,02/24/2021,11849,America's Supply Chains,NaN,14017,joe_biden
1,86 FR 11855,2021-04281,11856,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-03...,Presidential Document,Executive Order,03/01/2021,02/24/2021,11855,Revocation of Certain Presidential Actions,NaN,14018,joe_biden
2,86 FR 11089,2021-03874,11090,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-02...,Presidential Document,Executive Order,02/23/2021,02/17/2021,11089,Revocation of Executive Order 13801,NaN,14016,joe_biden
3,86 FR 10007,2021-03424,10009,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-02...,Presidential Document,Executive Order,02/18/2021,02/14/2021,10007,Establishment of the White House Office of Fai...,NaN,14015,joe_biden
4,86 FR 9429,2021-03139,9432,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-02...,Presidential Document,Executive Order,02/12/2021,02/10/2021,9429,Blocking Property With Respect to the Situatio...,NaN,14014,joe_biden
5,86 FR 8839,2021-02804,8844,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-02...,Presidential Document,Executive Order,02/09/2021,02/04/2021,8839,Rebuilding and Enhancing Programs To Resettle ...,NaN,14013,joe_biden
6,86 FR 8267,2021-02561,8271,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-02...,Presidential Document,Executive Order,02/05/2021,02/02/2021,8267,Creating a Comprehensive Regional Framework To...,NaN,14010,joe_biden
7,86 FR 8273,2021-02562,8275,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-02...,Presidential Document,Executive Order,02/05/2021,02/02/2021,8273,Establishment of Interagency Task Force on the...,NaN,14011,joe_biden
8,86 FR 8277,2021-02563,8280,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-02...,Presidential Document,Executive Order,02/05/2021,02/02/2021,8277,Restoring Faith in Our Legal Immigration Syste...,NaN,14012,joe_biden
9,86 FR 7793,2021-02252,7795,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-02...,Presidential Document,Executive Order,02/02/2021,01/28/2021,7793,Strengthening Medicaid and the Affordable Care...,NaN,14009,joe_biden


In [52]:
biden_df.loc[:, 'order_text'] = biden_df.html_url.apply(lambda x: gen_text(x))

In [54]:
biden_file = '/Users/lilygrier/Documents/Grad_School/Computational Content Analysis/biden_mar_7_21.csv'

In [55]:
biden_df.to_csv(biden_file)

In [64]:
full_exec_orders = pd.concat([not_null, null_orders, biden_df], axis=0, ignore_index=True)

In [65]:
# delete two clinton orders that didn't have text readily available
full_exec_orders[full_exec_orders.order_text.isnull()]

,Unnamed: 0,citation,document_number,end_page,html_url,pdf_url,type,subtype,publication_date,signing_date,start_page,title,disposition_notes,executive_order_number,president,order_text
900,680.0,63 FR 27651,98-13552,27654,https://www.federalregister.gov/documents/1998...,https://www.govinfo.gov/content/pkg/FR-1998-05...,Presidential Document,Executive Order,05/19/1998,05/14/1998,27651,Federalism,"Supplements: EO 12866, September 30, 1993; EO ...",13083.0,william_j_clinton,None
949,729.0,62 FR 7131,97-3992,7132,https://www.federalregister.gov/documents/1997...,https://www.govinfo.gov/content/pkg/FR-1997-02...,Presidential Document,Executive Order,02/14/1997,02/11/1997,7131,Advisory Committee on High-Performance Computi...,"Amended by: EO 13092, July 24, 1998; EO 13113,...",13035.0,william_j_clinton,None


In [66]:
full_exec_orders = full_exec_orders[full_exec_orders.order_text.notnull()]

In [67]:
full_file = '/Users/lilygrier/Documents/Grad_School/Computational Content Analysis/full_exec_orders_text.csv'

In [68]:
full_exec_orders.to_csv(full_file)

In [69]:
full_exec_orders

,Unnamed: 0,citation,document_number,end_page,html_url,pdf_url,type,subtype,publication_date,signing_date,start_page,title,disposition_notes,executive_order_number,president,order_text
0,0.0,82 FR 8115,2017-01623,8129,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-01...,Presidential Document,Executive Order,01/23/2017,01/17/2017,8115,"Amending the Civil Service Rules, Executive Or...","Amends: EO 13467, June 30, 2008; EO 13488, Jan...",13764.0,barack_obama,\n\nTitle 3—\nThe President\n\nExecutive Order...
1,1.0,82 FR 7619,2017-01487,7620,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-01...,Presidential Document,Executive Order,01/19/2017,01/13/2017,7619,Providing an Order of Succession Within the De...,"Revokes: EO 13557, November 4, 2010\nRevoked b...",13762.0,barack_obama,"\n\n\nExecutive Order 13762 of January 13, 201..."
2,2.0,82 FR 7621,2017-01489,7622,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-01...,Presidential Document,Executive Order,01/19/2017,01/13/2017,7621,Providing an Order of Succession Within the En...,"Revokes: EO 13737, August 12, 2016",13763.0,barack_obama,"\n\n\nExecutive Order 13763 of January 13, 201..."
3,3.0,82 FR 5331,2017-01197,5333,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-01...,Presidential Document,Executive Order,01/18/2017,01/13/2017,5331,Recognizing Positive Actions by the Government...,"Revokes in part: EO 13067, November 3, 1997\nR...",13761.0,barack_obama,\n\nTitle 3—\nThe President\n\nExecutive Order...
4,4.0,82 FR 5321,2017-01164,5322,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-01...,Presidential Document,Executive Order,01/17/2017,01/12/2017,5321,Amending Executive Order 11016 To Update Eligi...,"Amends: EO 11016, April 25, 1962",13758.0,barack_obama,\n\nTitle 3—\nThe President\n\nExecutive Order...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,NaN,86 FR 7029,2021-01762,7035,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-01...,Presidential Document,Executive Order,01/25/2021,01/20/2021,7029,Ethics Commitments by Executive Branch Personnel,NaN,13989.0,joe_biden,\n\nTitle 3—\nThe President\n\nExecutive Order...
1072,NaN,86 FR 7037,2021-01765,7043,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-01...,Presidential Document,Executive Order,01/25/2021,01/20/2021,7037,Protecting Public Health and the Environment a...,NaN,13990.0,joe_biden,"\n\n\nExecutive Order 13990 of January 20, 202..."
1073,NaN,86 FR 7045,2021-01766,7048,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-01...,Presidential Document,Executive Order,01/25/2021,01/20/2021,7045,Protecting the Federal Workforce and Requiring...,NaN,13991.0,joe_biden,"\n\n\nExecutive Order 13991 of January 20, 202..."
1074,NaN,86 FR 7049,2021-01767,7050,https://www.federalregister.gov/documents/2021...,https://www.govinfo.gov/content/pkg/FR-2021-01...,Presidential Document,Executive Order,01/25/2021,01/20/2021,7049,Revocation of Certain Executive Orders Concern...,NaN,13992.0,joe_biden,"\n\n\nExecutive Order 13992 of January 20, 202..."
